In [1]:
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import gymnasium as gym
import numpy as np
import globals as gl
import classes as cl
import time
import os
# from gymnasium_env import ContinousPricingGame
from Environment import ConPricingGame
import torch

In [2]:
gl.initialize()

num_procs=6
model = SAC
model_name="CoSAC_MP"
timesteps= 1_000_000
num_timesteps=10
state_onehot=False
costs=[gl.LOW_COST, gl.HIGH_COST]
adv_mixed_strategy = cl.MixedStrategy(strategiesList=[cl.Strategy(
        cl.StrategyType.static, NNorFunc=cl.myopic, name="myopic")], probablitiesArray=[1])

In [3]:
lrs=[0.00005,0.00008, 0.0003]

In [4]:
# def make_env(rank, seed=0):
#     """
#     Utility function for multiprocessed env.
#     :param seed: (int) the inital seed for RNG
#     :param rank: (int) index of the subprocess
#     """

#     def _init():
#         env = ContinousPricingGame(tuple_costs=costs, adversary_mixed_strategy=adv_mixed_strategy, state_onehot=state_onehot)
#         env.reset(seed=seed + rank)
#         # use a seed for reproducibility
#         # Important: use a different seed for each environment
#         # otherwise they would generate the same experiences
#         return env

#     set_random_seed(seed)
#     return _init

In [5]:
def run(lr=None):    
    seed=int(time.time())

    iter_name = f"{model_name}-{str(seed)}"
    models_dir = os.path.join("models", iter_name)
    log_dir = os.path.join("logs", iter_name)
             
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)

    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
             
    train_env = make_vec_env(ConPricingGame, n_envs=num_procs, seed=seed, vec_env_cls=SubprocVecEnv,env_kwargs=\
                           dict(tuple_costs = costs,adversary_mixed_strategy = adv_mixed_strategy))
#     train_env = make_vec_env(ContinousPricingGame, n_envs=num_procs, seed=0, vec_env_cls=SubprocVecEnv,\
#                              env_kwargs=dict(tuple_cost=costs))

    lr_=(gl.LR if (lr is None) else lr)
#     train_env = ContinousPricingGame()
#     train_env.reset()
    
#     train_env.reset()
    model_ = model('MlpPolicy', train_env,learning_rate=lr_,verbose=0, tensorboard_log=log_dir, gamma=gl.GAMMA)
    
    start=time.time()
    for i in range(num_timesteps):
        model_.learn(total_timesteps=timesteps,
                     reset_num_timesteps=False,tb_log_name=iter_name)
    model_.save(os.path.join(models_dir, str(timesteps*i)))
    running_time=time.time()- start

    # test and write results
    env = ConPricingGame(tuple_costs=costs, adversary_mixed_strategy=adv_mixed_strategy)
    for iter in range(gl.NUM_STOCHASTIC_ITER):
             
        obs,_ = env.reset()
        done = False

        actions = []
        while not done:
            action, _states = model_.predict(obs)
            obs, reward, done,trunc, info = env.step(action)

            actions.append(int(action))
        # name	ep	costs	adversary	agent_returnC:\Users\sjaha\Desktop\RL\Gym\stableBaseline3\environment.py	adv_return	agent_rewards	actions	agent_prices	\
        # adv_prices	agent_demands	adv_demands	lr	hist	total_stages	action_step	num_actions	gamma	\
        # stae_onehot	seed	num_procs	running_time
        data=[iter_name, timesteps*num_timesteps,("L" if (costs[0]<costs[1]) else "H"), env.adversary_strategy.name,\
              sum(env.profit[0]), sum(env.profit[1]),  str(env.profit[0]), str(actions), str(env.prices[0]),\
              str(env.prices[1]), str(env.demand_potential[0]),str(env.demand_potential[1]), lr_, gl.NUM_ADV_HISTORY,\
              gl.TOTAL_STAGES, gl.ACTION_STEP, gl.NUM_ACTIONS, gl.GAMMA, False, seed, num_procs, running_time]
        cl.write_to_excel(data)
    print(lr, "=lr completed")


In [ ]:
if __name__ == "__main__":
    for lr in lrs:

        for _ in range(2):
            run(lr=lr)
        

5e-05 =lr completed
5e-05 =lr completed
8e-05 =lr completed
8e-05 =lr completed
